In [1]:
import pandas as pd
pd.set_option("display.max.columns", None)  # Show all the columns
pd.options.display.float_format = '{:20,.2f}'.format #Get rid of the scientific notation
pd.set_option('display.max.columns', None)


import json      # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs

import geopandas as gpd

import pprint    # library for cleanly printing Python data structures
pp = pprint.PrettyPrinter()

In [2]:
typesDict = {'CityFIPS': str, 'TractFIPS': str}

df = pd.read_csv('../500 Cities Data/500_Cities__Local_Data_for_Better_Health__2019_release.csv', dtype=typesDict)
type(df)
df.head()

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,DataValueTypeID,Data_Value_Type,Data_Value,Low_Confidence_Limit,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
0,2016,IL,Illinois,Chicago,Census Tract,BRFSS,Health Outcomes,1714000-17031612000,All teeth lost among adults aged >=65 Years,%,CrdPrv,Crude prevalence,31.60,24.50,39.90,NaN,NaN,1545,"(41.7978467034, -87.6525358919)",HLTHOUT,TEETHLOST,1714000,17031612000,Teeth Loss
1,2016,IL,Illinois,Chicago,Census Tract,BRFSS,Prevention,1714000-17031660700,Visits to dentist or dental clinic among adult...,%,CrdPrv,Crude prevalence,41.90,37.90,46.30,NaN,NaN,2366,"(41.7738453587, -87.6810954614)",PREVENT,DENTAL,1714000,17031660700,Dental Visit
2,2017,IL,Illinois,Decatur,Census Tract,BRFSS,Health Outcomes,1718823-17115001100,Current asthma among adults aged >=18 Years,%,CrdPrv,Crude prevalence,10.00,9.50,10.50,NaN,NaN,3485,"(39.8496654386, -88.9073271617)",HLTHOUT,CASTHMA,1718823,17115001100,Current Asthma
3,2016,IL,Illinois,Joliet,Census Tract,BRFSS,Health Outcomes,1738570-17197881603,All teeth lost among adults aged >=65 Years,%,CrdPrv,Crude prevalence,21.00,15.60,27.40,NaN,NaN,3301,"(41.5405415324, -88.133133385)",HLTHOUT,TEETHLOST,1738570,17197881603,Teeth Loss
4,2017,IL,Illinois,Elgin,Census Tract,BRFSS,Health Outcomes,1723074-17031804405,High cholesterol among adults aged >=18 Years ...,%,CrdPrv,Crude prevalence,25.70,25.00,26.30,NaN,NaN,3632,"(42.0349418585, -88.2462502287)",HLTHOUT,HIGHCHOL,1723074,17031804405,High Cholesterol


In [3]:
df.columns #column headers

Index(['Year', 'StateAbbr', 'StateDesc', 'CityName', 'GeographicLevel',
       'DataSource', 'Category', 'UniqueID', 'Measure', 'Data_Value_Unit',
       'DataValueTypeID', 'Data_Value_Type', 'Data_Value',
       'Low_Confidence_Limit', 'High_Confidence_Limit',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'PopulationCount',
       'GeoLocation', 'CategoryID', 'MeasureId', 'CityFIPS', 'TractFIPS',
       'Short_Question_Text'],
      dtype='object')

## What Kind Data is Available?

In [4]:

#df.Category.unique()
#df.Measure.unique()
#df.Measure.value_counts()
#df['Measure'].value_counts()
# df['Measure'].head()

columnsWeCareAbout = ['Year','StateAbbr','CityName', 'TractFIPS','Measure','MeasureId','GeographicLevel','Data_Value_Type','Data_Value', 'Data_Value_Unit','DataSource', 'PopulationCount']
# columnsWeCareAbout = ['Year', 'StateAbbr', 'StateDesc', 'CityName', 'GeographicLevel',
#        'DataSource', 'Category', 'UniqueID', 'Measure', 'Data_Value_Unit',
#        'DataValueTypeID', 'Data_Value_Type', 'Data_Value',
#        'Low_Confidence_Limit', 'High_Confidence_Limit',
#        'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'PopulationCount',
#        'GeoLocation', 'CategoryID', 'MeasureId', 'CityFIPS', 'TractFIPS',
#        'Short_Question_Text']


tractDF = df[columnsWeCareAbout].copy()
tractDF = tractDF.loc[tractDF['GeographicLevel']=='Census Tract']

# tractDF['DataValueType'] = tractDF['Data_Value_Type']
# tractDF['DataValue'] = tractDF['Data_Value']
# tractDF = tractDF[columnsWeCareAbout]

print('MeasuresIds:')
print(tractDF.MeasureId.unique())

print('\n','Measures:')
print(tractDF.Measure.unique())

print('\n', 'Data_Value_Types:')
print(tractDF.Data_Value_Type.unique(), '\n')

print('\n', 'Data_Value_Units:')
print(tractDF.Data_Value_Unit.unique(), '\n')

print('\n', len(tractDF), ' rows.')

#The following equality ensures that we didn't lose any Measure's when we filtered for only census-level data
if not (len(tractDF.Measure.unique()) == len(df.MeasureId.unique())):
    raise ValueError('something bad happened')

MeasuresIds:
['TEETHLOST' 'DENTAL' 'CASTHMA' 'HIGHCHOL' 'SLEEP' 'DIABETES' 'OBESITY'
 'CANCER' 'STROKE' 'MHLTH' 'CSMOKING' 'CHOLSCREEN' 'PAPTEST' 'CHD'
 'CHECKUP' 'KIDNEY' 'BINGE' 'LPA' 'ARTHRITIS' 'MAMMOUSE' 'BPMED'
 'COLON_SCREEN' 'PHLTH' 'COREW' 'BPHIGH' 'COREM' 'ACCESS2' 'COPD']

 Measures:
['All teeth lost among adults aged >=65 Years'
 'Visits to dentist or dental clinic among adults aged >=18 Years'
 'Current asthma among adults aged >=18 Years'
 'High cholesterol among adults aged >=18 Years who have been screened in the past 5 Years'
 'Sleeping less than 7 hours among adults aged >=18 Years'
 'Diagnosed diabetes among adults aged >=18 Years'
 'Obesity among adults aged >=18 Years'
 'Cancer (excluding skin cancer) among adults aged >=18 Years'
 'Stroke among adults aged >=18 Years'
 'Mental health not good for >=14 days among adults aged >=18 Years'
 'Current smoking among adults aged >=18 Years'
 'Cholesterol screening among adults aged >=18 Years'
 'Papanicolaou smear use amo

## Define measure and year

In [5]:
myMeasure = 'KIDNEY'
myYear = 2017
outFile = 'df_factory_outputs/cynthiasKidneyDF.csv'

tractDF.loc[tractDF['MeasureId'] == myMeasure].head()

,Year,StateAbbr,CityName,TractFIPS,Measure,MeasureId,GeographicLevel,Data_Value_Type,Data_Value,Data_Value_Unit,DataSource,PopulationCount
28,2017,IL,Peoria,17143003300,Chronic kidney disease among adults aged >=18 ...,KIDNEY,Census Tract,Crude prevalence,3.60,%,BRFSS,3024
38,2017,IL,Decatur,17115002401,Chronic kidney disease among adults aged >=18 ...,KIDNEY,Census Tract,Crude prevalence,3.10,%,BRFSS,2597
63,2017,IL,Chicago,17031151002,Chronic kidney disease among adults aged >=18 ...,KIDNEY,Census Tract,Crude prevalence,2.70,%,BRFSS,4567
85,2017,IL,Joliet,17197882702,Chronic kidney disease among adults aged >=18 ...,KIDNEY,Census Tract,Crude prevalence,2.80,%,BRFSS,3075
152,2017,IL,Decatur,17115001700,Chronic kidney disease among adults aged >=18 ...,KIDNEY,Census Tract,Crude prevalence,3.40,%,BRFSS,2085


In [6]:
#for historical reasons, the dataframe of interest is named obDF (originally for obesityDF)
obDF = tractDF.loc[tractDF['MeasureId'] == myMeasure].copy()
obDF = obDF.loc[obDF['Year'] == myYear]

# len(obDF)
print(len(obDF.TractFIPS.unique()), ' unique FIPS codes')
print(len(obDF), ' total rows')
print("Some tracts' data is split across multiple entries.")

print('\n Number of rows missing TractFIPS: ', obDF.TractFIPS.isnull().sum())
print('\n Number of FIPS appearing in multiple rows: ',len(obDF[obDF.duplicated(['TractFIPS'], keep='first')]))  #THERE APPEAR TO BE 356 DUPLICATED FIPS CODES

print('\n data sources:', obDF.DataSource.unique())

obDF = obDF[obDF.Data_Value.notnull()]  #drop all rows which have NaN for DataValue

27648  unique FIPS codes
28004  total rows
Some tracts' data is split across multiple entries.

 Number of rows missing TractFIPS:  0

 Number of FIPS appearing in multiple rows:  356

 data sources: ['BRFSS']


In [7]:
#Lets define a function which takes a 2-line dataframe, combines the rows, and returns a 1-line dataframe
def mergeTracts(df):
    '''df is 2-row DataFrame that needs to merged down to 1 row.
       returns a new Series with a single, merged row.
       Merge logic may need to be decided by Data_Value_Unit'''
        
    mergeDict = {'popCount':[], 'dataValue':[]}
    shouldBeIdenticalDict = {}
    shouldBeIdentical = ['TractFIPS', 'Measure', 'GeographicLevel', 'Data_Value_Type', 'Data_Value_Unit', 'DataSource', 'Year']
    
    for idx, row in df.iterrows():
        
        #Error Checking - Make sure that columns that should be identical are
        for col in shouldBeIdentical:
            if col not in shouldBeIdenticalDict.keys():
                shouldBeIdenticalDict[col] = row[col]
            elif shouldBeIdenticalDict[col] != row[col]:
                print('Column ' + col + ' should be the same across two rows but it is not!')
                raise ValueError
        
        mergeDict['popCount'].append(row['PopulationCount'])
        mergeDict['dataValue'].append(row['Data_Value'])
       #At this point, our mergedict contains the data we need to merge in ordered lists.
    
    #This merging logic probably only applies to % columns    
    if df['Data_Value_Unit'].unique()[0] == '%':
        totalPopAffected = 0
        for i in range(len(mergeDict['popCount'])):
            totalPopAffected += (mergeDict['popCount'][i] * mergeDict['dataValue'][i])
        totalPop = sum(mergeDict['popCount'])
        percentTotal = totalPopAffected / totalPop
    else:
        print('Unexpected Data_Value_Unit: ', df['Data_Value_Unit'].unique()[0])
        raise ValueError
        
    #now build a new row using these merged data values
    
    new_row = df.iloc[0].copy()
    new_row['Data_Value'] = percentTotal
    new_row['PopulationCount'] = totalPop
        
    return new_row
    

# tst = obDF.loc[obDF['TractFIPS'] == '1703180410200']                
# mergeTracts(tst).head()

In [8]:
obDF['isDuplicate'] = obDF['TractFIPS'].duplicated() #returns a series of True/False
obDF_duplicated = obDF.loc[obDF['isDuplicate'] == True].copy()

allMergedDF = pd.DataFrame(columns=obDF.columns)
mergedFIPSList = []

#Here we merge all the tracts which need it.
for i in range(0, len(obDF_duplicated)):
    curFIPS = obDF_duplicated.iloc[i]['TractFIPS']
    
    if curFIPS in mergedFIPSList:
        print("ALREADY MERGED " + str(curFIPS)) #this FIPS code was split across 3 records
        continue
     
    splitTractDF = obDF.loc[obDF['TractFIPS'] == curFIPS].copy() #this should return a dataframe with the rows for a each specific duplicated FIPS
    
    mergedRow = mergeTracts(splitTractDF)
    allMergedDF = allMergedDF.append(mergeTracts(splitTractDF))
    mergedFIPSList.append(curFIPS)
    
allMergedDF = allMergedDF.append(mergedRow)
obDFDuplicatesRemoved = obDF.loc[~obDF.duplicated('TractFIPS', keep=False)]
obDFFinal = obDFDuplicatesRemoved.append(allMergedDF, sort=True)
    
print('Length of DF with duplicates: ', len(obDF))
print('\n Length of DF with all duplicates removed: ', len(obDFDuplicatesRemoved))
print('\n Length of DF with merged rows added back: ', len(obDFFinal))

obDFFinal = obDFFinal.drop(columns='isDuplicate')

obDFFinal.head(30)

ALREADY MERGED 48157670700
ALREADY MERGED 06059099203
Length of DF with duplicates:  27210

 Length of DF with all duplicates removed:  26728

 Length of DF with merged rows added back:  26969


,CityName,DataSource,Data_Value,Data_Value_Type,Data_Value_Unit,GeographicLevel,Measure,MeasureId,PopulationCount,StateAbbr,TractFIPS,Year
28,Peoria,BRFSS,3.60,Crude prevalence,%,Census Tract,Chronic kidney disease among adults aged >=18 ...,KIDNEY,3024,IL,17143003300,2017
38,Decatur,BRFSS,3.10,Crude prevalence,%,Census Tract,Chronic kidney disease among adults aged >=18 ...,KIDNEY,2597,IL,17115002401,2017
63,Chicago,BRFSS,2.70,Crude prevalence,%,Census Tract,Chronic kidney disease among adults aged >=18 ...,KIDNEY,4567,IL,17031151002,2017
85,Joliet,BRFSS,2.80,Crude prevalence,%,Census Tract,Chronic kidney disease among adults aged >=18 ...,KIDNEY,3075,IL,17197882702,2017
152,Decatur,BRFSS,3.40,Crude prevalence,%,Census Tract,Chronic kidney disease among adults aged >=18 ...,KIDNEY,2085,IL,17115001700,2017
162,Chicago,BRFSS,4.50,Crude prevalence,%,Census Tract,Chronic kidney disease among adults aged >=18 ...,KIDNEY,1956,IL,17031670400,2017
163,Chicago,BRFSS,4.20,Crude prevalence,%,Census Tract,Chronic kidney disease among adults aged >=18 ...,KIDNEY,5936,IL,17031750500,2017
195,Chicago,BRFSS,1.90,Crude prevalence,%,Census Tract,Chronic kidney disease among adults aged >=18 ...,KIDNEY,1928,IL,17031030103,2017
198,Evanston,BRFSS,2.10,Crude prevalence,%,Census Tract,Chronic kidney disease among adults aged >=18 ...,KIDNEY,3955,IL,17031808800,2017
233,Chicago,BRFSS,4.10,Crude prevalence,%,Census Tract,Chronic kidney disease among adults aged >=18 ...,KIDNEY,1761,IL,17031836100,2017


In [9]:
obDFFinal.to_csv(outFile)